# Visualisations - Neater Implementation

In [91]:
# Enable Intellisense
%config IPCompleter.greedy=True

import pandas as pd
import numpy as np
import io
import requests
import seaborn as sns
from matplotlib import pyplot, dates
import plotly.express as px 
import datetime
import plotly.graph_objects as go
import plotly.io as pio
import plotly.offline
sns.set()

# Pie Chart

In [92]:
def create_pie_chart(df, values, names, save_file_name="", color_discrete_sequence=[]):
    if (color_discrete_sequence==[]):
        fig = px.pie(df, values=values, names=names)
    else:
        fig = px.pie(df, values=values, names=names, color_discrete_sequence=color_discrete_sequence)
    fig.update_layout(
        legend=dict(x=0, y=1), 
        showlegend=False,
        margin=dict(
            t=0, # 50
            b=20,
            l=0,
            r=0,
        ),
    )
    fig.update_traces(hoverinfo= 'label+percent', 
                      hovertemplate = '%{label}<br>%{value}',
                      textinfo='value+label',
                      textposition='inside')
        
    if save_file_name != "":
        plotly.offline.plot(fig, filename = save_file_name, auto_open=True, 
                            config = dict(displayModeBar=False))
        
    return fig

## Recoveries Per Province

In [93]:
prov_recov_totals = pd.read_csv('data/tot_recovered_provinces.csv')
prov_recov_totals

,province,total,latest_change
0,Eastern Cape,839,0
1,Free State,108,0
2,Gauteng,1657,74
3,KwaZulu-Natal,818,1
4,Limpopo,37,0
5,Mpumalanga,53,4
6,Northern Cape,27,3
7,North West,28,0
8,Western Cape,3731,210
9,Unknown,0,0


### Total

In [94]:
create_pie_chart(prov_recov_totals, 'total', 'province', 'tot_recovered_per_province.html')

### Latest Change

In [95]:
create_pie_chart(prov_recov_totals, 'latest_change', 'province', 
                 'latest_change_recovered_per_province.html')

## Confirmed Per Province

In [96]:
prov_totals_data = pd.read_csv('data/tot_provinces.csv')
prov_totals_data

,province,total,latest_change
0,Eastern Cape,2052,116
1,Free State,163,5
2,Gauteng,2343,14
3,KwaZulu-Natal,1567,24
4,Limpopo,88,11
5,Mpumalanga,76,5
6,Northern Cape,37,0
7,North West,72,2
8,Western Cape,10035,741
9,Unknown,0,0


In [97]:
create_pie_chart(prov_totals_data, 'total', 'province', 'tot_cases_per_province.html')

### Latest Change

In [98]:
create_pie_chart(prov_totals_data, 'latest_change', 'province', 'latest_change_cases_per_province.html')

## Deaths Per Province

In [99]:
prov_deaths_totals_data = pd.read_csv('data/tot_deaths_provinces.csv')
prov_deaths_totals_data

,province,total,latest_change
0,Eastern Cape,39,4
1,Free State,6,0
2,Gauteng,26,1
3,KwaZulu-Natal,45,0
4,Limpopo,3,0
5,Mpumalanga,0,0
6,Northern Cape,0,0
7,North West,1,0
8,Western Cape,166,17
9,Unknown,0,0


In [100]:
create_pie_chart(prov_deaths_totals_data, 'total', 'province', 'tot_deaths_per_province.html')

### Latest Change

In [101]:
create_pie_chart(prov_deaths_totals_data, 'latest_change', 'province', 'latest_change_deaths_per_province.html')

## Tests Per Province

In [102]:
prov_tests_totals_data = pd.read_csv('data/tot_tests_provinces.csv')
prov_tests_totals_data

,province,total
0,Eastern Cape,38122
1,Free State,19265
2,Gauteng,127030
3,KwaZulu-Natal,67853
4,Limpopo,8239
5,Mpumalanga,11414
6,Northern Cape,3683
7,North West,5812
8,Western Cape,82865
9,Unknown,22069


In [103]:
create_pie_chart(prov_tests_totals_data, 'total', 'province', 'tot_tests_per_province.html')

## Deaths Vs Recoveries

In [104]:
recovered_deaths_totals = pd.read_csv('data/gen_data.csv', 
                                      usecols=['tot_recoveries','tot_deaths']).reset_index()
recovered_deaths_totals.rename({"tot_recoveries":"Recoveries", "tot_deaths":"Deaths"}, axis=1, inplace=True)
recovered_deaths_totals = recovered_deaths_totals.melt(id_vars=['index'], var_name='variable', 
                                                    value_name='total')

# Below is needed due to integers losing accuracy in division
recovered_deaths_totals.drop('index', axis=1, inplace=True)
recovered_deaths_totals['total'] = recovered_deaths_totals['total'].apply(lambda x: str(x).replace(" ", ""))
recovered_deaths_totals['total'] = pd.to_numeric(recovered_deaths_totals['total'], downcast='float')
recovered_deaths_totals

,variable,total
0,Deaths,286.0
1,Recoveries,7298.0


In [105]:
create_pie_chart(recovered_deaths_totals, 'total', 'variable', 'tot_recovered_deaths.html', 
                 ['green','black'])

# Line Graph
___

In [106]:
def create_line_graph(data, # Dataframe
                      xaxis_title, yaxis_title, # Strings
                      x_col, # string denoting column in dataframe for x axis 
                      y_cols, # list of strings denoting column in dataframe for y values for each curve
                      names, # list of strings - names of each curve
                      colors, # list of strings - color of each curve
                      annotations =[],
                      start_x="",
                      date_format = '%Y-%m-%d',
                      html_file_name = "",
                     ):
    
    data = data.copy()
    
    if start_x != "":
        start_index = data.loc[data[x_col]==start_x,x_col].index[0]
        data.drop(data.index[:start_index], inplace=True)
#         data = data.iloc[start_index:]
    
    if x_col == "date":
        data['date'] = pd.to_datetime(data['date'], format=date_format)
    
    data_long = pd.melt(data, id_vars=[x_col], value_vars=y_cols) 
    fig = px.line(data_long, x='date', y='value', color='variable', color_discrete_sequence=colors)
    
    fig.update_layout(
        annotations = annotations,
        title="",
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        hovermode= 'x',
        legend=dict(x=0.01, y=.98, title=dict(text="")),
        xaxis=dict(fixedrange=True),
        yaxis=dict(fixedrange=True),
        margin=dict(
            t=0,
            b=0,
            l=0,
            r=0,
        ),
    )
    
    fig.update_traces(mode='lines')
    
    for i in range(0, len(y_cols)):
        fig.data[i].name = names[i]
    
        fig.data[i].hovertemplate = '%{y}'
        fig.data[i].hoverlabel.namelength = 0
        
    if html_file_name != "":
        plotly.offline.plot(fig, filename = html_file_name, auto_open=True, 
                            config = dict(displayModeBar=False))
    
    return fig

## Cumulative Data

In [107]:
all_cum_data = pd.read_csv('data/all_cum_data.csv')
all_cum_data

,date,confirmed,tests,deaths,recovered,active,confirmed_div_by_tests,deaths_div_by_confirmed,recovered_div_by_confirmed,confirmed_per_mil,tests_per_mil,deaths_per_mil,recovered_per_mil,active_per_mil
0,2020-02-11,0,61,0,0,0,0.000,0.000,0.000,0.00,1.03,0.00,0.00,0.00
1,2020-02-13,0,67,0,0,0,0.000,0.000,0.000,0.00,1.13,0.00,0.00,0.00
2,2020-02-14,0,71,0,0,0,0.000,0.000,0.000,0.00,1.20,0.00,0.00,0.00
3,2020-02-19,0,95,0,0,0,0.000,0.000,0.000,0.00,1.60,0.00,0.00,0.00
4,2020-02-20,0,106,0,0,0,0.000,0.000,0.000,0.00,1.79,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,2020-05-14,12739,403018,238,5676,6825,0.032,0.019,0.446,215.20,6808.23,4.02,95.89,115.30
79,2020-05-15,13524,421555,247,6083,7194,0.032,0.018,0.450,228.46,7121.38,4.17,102.76,121.53
80,2020-05-16,14355,439559,261,6478,7616,0.033,0.018,0.451,242.50,7425.52,4.41,109.43,128.66
81,2020-05-17,15515,460873,264,7006,8245,0.034,0.017,0.452,262.10,7785.58,4.46,118.35,139.28


In [108]:
# start_index = all_cum_data.set_index('date').index.get_loc("2020-05-06")
# all_cum_data.iloc[70:]

In [109]:
all_cum_data.loc[all_cum_data['date']=='2020-05-02','date'].index[0]

66

### Tests & Confirmed

In [110]:
create_line_graph(all_cum_data, "Date", "Cumulative No", 
                 x_col = 'date', y_cols = ['tests','confirmed'], 
                 names = ['Tests','Positive Cases'], colors = ['blue', 'firebrick'],
                 html_file_name="date_vs_cases_tests.html"
                 )

### Confirmed
TODO - Add annotation for first day of lockdown

In [111]:
first_day_lockdown_annotations = [dict(
            x=pd.to_datetime('2020-03-27', format='%Y/%m/%d'),
            y=1170,
            xref="x",
            yref="y",
            text="First Day of Lockdown",
            showarrow=True,
            arrowsize=1.5,
            arrowhead=1,
            yshift=4,
            ax=0,
            ay=-40
        )]

create_line_graph(all_cum_data, "Date", "Cumulative No of Confirmed Cases", 
                 x_col = 'date', y_cols = ['confirmed'], 
                 names = ['Confirmed Cases'], colors = ['firebrick'],
                 annotations=first_day_lockdown_annotations,
                 start_x = "2020-03-03",
                 html_file_name='date_vs_cases.html'
                 )

### Recovered & Deaths

In [112]:
create_line_graph(all_cum_data, "Date", "Cumulative No", 
                 x_col = 'date', y_cols = ['recovered','deaths'], 
                 names = ['Recoveries','Deaths'], colors = ['green', 'black'],
                 start_x = "2020-03-21",
                 html_file_name='cumulative_deaths_recovered.html'
                 )

### Deaths

In [113]:
create_line_graph(all_cum_data, "Date", "Cumulative No of Deaths", 
                 x_col = 'date', y_cols = ['deaths'], 
                 names = ['Deaths'], colors = ['black'],
                 start_x = "2020-03-26",
                 html_file_name='cumulative_deaths.html'
                 )

### Active Cases

In [114]:
create_line_graph(all_cum_data, "Date", "Active Cases", 
                 x_col = 'date', y_cols = ['active'], 
                 names = ['Active Cases'], colors = ['red'],
                 start_x = "2020-03-03",
                 html_file_name='date_vs_active.html'
                 )

### Confirmed divided by tests
**I.e. ratio of tests positive** <br>
confirmed_div_by_tests

In [115]:
create_line_graph(all_cum_data, "Date", "Ratio of Confirmed Cases to Tests Conducted", 
                 x_col = 'date', y_cols = ['confirmed_div_by_tests'], 
                 names = ['Confirmed Cases : Tests'], colors = ['firebrick'],
                 start_x = "2020-03-03",
                 html_file_name="date_vs_confirmed_div_by_tests.html"
                 )

### Deaths Div by Confirmed & Recovered Div by Confirmed

In [116]:
create_line_graph(all_cum_data, "Date", "Total No", 
                 x_col = 'date', y_cols = ['recovered_div_by_confirmed','deaths_div_by_confirmed'], 
                 names = ['Recovered Divided by Confirmed', 'Deaths Divided by Confirmed'], 
                 colors = ['green','black'],
                 start_x = "2020-03-21"
                 )

## Daily Data

In [117]:
all_daily_data = pd.read_csv('data/all_daily_data.csv')
all_daily_data

,date,confirmed,tests,deaths,recovered,active
0,2020-02-11,0,61,0,0,0
1,2020-02-13,0,6,0,0,0
2,2020-02-14,0,4,0,0,0
3,2020-02-19,0,24,0,0,0
4,2020-02-20,0,11,0,0,0
...,...,...,...,...,...,...
78,2020-05-14,665,16666,19,931,-285
79,2020-05-15,785,18537,9,407,369
80,2020-05-16,831,18004,14,395,422
81,2020-05-17,1160,21314,3,528,629


### Tests & Positive Cases

In [118]:
create_line_graph(all_daily_data, "Date", "Daily Change", 
                 x_col = 'date', y_cols = ['tests','confirmed'], 
                 names = ['Tests','Confirmed Cases'], colors = ['blue', 'firebrick'],
                 html_file_name="date_vs_daily_tests_cases.html"
                 )

### Positive Cases

In [119]:
create_line_graph(all_daily_data, "Date", "Daily Change in Confirmed Cases", 
                 x_col = 'date', y_cols = ['confirmed'], 
                 names = ['Confirmed Cases'], colors = ['firebrick'],
                 start_x = "2020-03-03",
                 html_file_name='date_vs_daily_cases.html'
                 )

### Deaths & Recoveries

In [120]:
create_line_graph(all_daily_data, "Date", "Daily Change", 
                 x_col = 'date', y_cols = ['recovered','deaths'], 
                 names = ['Recoveries','Deaths'], colors = ['green', 'black'],
                 start_x = "2020-03-21",
                 html_file_name='daily_deaths_recovered.html'
                 )

### Deaths

In [121]:
create_line_graph(all_daily_data, "Date", "Daily Change in Deaths", 
                 x_col = 'date', y_cols = ['deaths'], 
                 names = ['Deaths'], colors = ['black'],
                 start_x = "2020-03-26",
                 html_file_name='daily_deaths.html'
                 )

# Line Graph Per Province
___

In [122]:
def create_line_graph_group(data, # Dataframe
                      xaxis_title, yaxis_title, # Strings
                      x_col, # string denoting column in dataframe for x axis 
                      y_col, # strings denoting column in dataframe for y value of each group
                      group_col,
                      date_format = '%Y-%m-%d',
                      html_file_name = ""
                     ):
    if x_col == "date":
        data['date'] = pd.to_datetime(data['date'], format=date_format)
    
    fig = px.line(data, x=x_col, y=y_col, color=group_col,
                         hover_data=[group_col])
    fig.update_layout(
        title="",
        xaxis_title=xaxis_title,
        yaxis_title=yaxis_title,
        hovermode= 'x',
        legend=dict(x=0.01, y=.98, title=dict(text="")),
        xaxis=dict(fixedrange=True),
        yaxis=dict(fixedrange=True),
        margin=dict(
            t=0,
            b=0,
            l=0,
            r=0,
        ),
    )
    
    for d in fig.data:
        d.hoverinfo = 'all',
        d.hovertemplate = '%{customdata[0]}<br>%{y}'
        d.hoverlabel.namelength = 0
        
    if html_file_name != "":
        plotly.offline.plot(fig, filename = html_file_name, auto_open=True, 
                            config = dict(displayModeBar=False))
        
    return fig
    

## Confirmed

In [123]:
confirmed_by_prov_timeline= pd.read_csv("data/confirmed_by_prov_timeline.csv")
confirmed_by_prov_timeline.head()

,date,province,cum_no,daily_no
0,2020-03-05,EC,0,0
1,2020-03-07,EC,0,0
2,2020-03-08,EC,0,0
3,2020-03-09,EC,0,0
4,2020-03-11,EC,0,0


### Total

In [124]:
create_line_graph_group(confirmed_by_prov_timeline, "Date", "Total Confirmed Cases",
                       x_col='date', y_col='cum_no', group_col='province',
                       html_file_name="date_vs_cases_per_province.html")

### Daily Change

In [125]:
create_line_graph_group(confirmed_by_prov_timeline, "Date", "Daily Change in Confirmed Cases",
                       x_col='date', y_col='daily_no', group_col='province', 
                       html_file_name="date_vs_daily_cases_per_province.html")

## Deaths

In [126]:
deaths_by_prov_timeline= pd.read_csv("data/deaths_by_prov_timeline.csv")
deaths_by_prov_timeline.head()

,date,province,cum_no,daily_no
0,2020-03-27,EC,0,0
1,2020-03-28,EC,0,0
2,2020-03-30,EC,0,0
3,2020-03-31,EC,0,0
4,2020-04-03,EC,0,0


### Total

In [127]:
create_line_graph_group(deaths_by_prov_timeline, "Date", "Total Deaths",
                       x_col='date', y_col='cum_no', group_col='province',
                       html_file_name='date_vs_deaths_per_province.html')

### Daily Change

In [128]:
create_line_graph_group(deaths_by_prov_timeline, "Date", "Daily Change in Deaths",
                       x_col='date', y_col='daily_no', group_col='province',
                       html_file_name="date_vs_daily_deaths_per_province.html")

## Recoveries

In [129]:
recoveries_by_prov_timeline= pd.read_csv("data/recoveries_by_prov_timeline.csv")
recoveries_by_prov_timeline.head()

,date,province,cum_no,daily_no
0,2020-04-17,EC,9,9
1,2020-04-18,EC,9,0
2,2020-04-19,EC,9,0
3,2020-04-20,EC,15,6
4,2020-04-21,EC,15,0


### Total

In [130]:
create_line_graph_group(recoveries_by_prov_timeline, "Date", "Total Recoveries",
                       x_col='date', y_col='cum_no', group_col='province',
                       html_file_name='date_vs_recoveries_per_province.html')

### Daily Change

In [131]:
create_line_graph_group(recoveries_by_prov_timeline, "Date", "Daily Change in Recoveries",
                       x_col='date', y_col='daily_no', group_col='province',
                       html_file_name="date_vs_daily_recoveries_per_province.html")